## SQL Table

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
# from sentence_transformers import SentenceTransformer
import os 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import FAISS
model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', 
                                            model_kwargs={'device':'cuda'})
# model = sentence_transformer( model_name ='sentence-transformers/all-MiniLM-L6-v2')
sample_encoding = model.embed_query("Sample Sentence")
print("Dimension of the Embedding : ", len(sample_encoding))

c:\Users\krish\anaconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dimension of the Embedding :  384


In [2]:
openai_embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002',
                                                       openai_api_key=os.getenv('OPENAI_KEY'))
sample_encoding = openai_embedding_model.embed_query("Sample Sentence")
print("Dimension of the Embedding : ", len(sample_encoding))

Dimension of the Embedding :  1536


In [13]:
import os
import pandas as pd
from dateutil.parser import parse
df = pd.read_csv('.\\Canon Open Source Data\\excel knowledge base.txt', delimiter='\t')
df.drop([0], axis=0, inplace=True)
df.drop('Field name', axis =1 , inplace=True)
df.rename(columns={'Record No.':'index'}, inplace=True)
df.drop('index', axis=1, inplace =True)
df.fillna("Not Available", inplace =True)
df['Date'] = [parse(df.loc[i].Date) for i in df.index.to_list()]

In [14]:
df

,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
1,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
2,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
3,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available
4,2023-06-29,PBC,ICB,East,#VALUE!,Product,Nikon & Sony started promoting Z8 and Zve1 vis...,Not Available
5,2023-06-22,CBC-PPP,PPP,India,#VALUE!,Market footfall,Konica Minolta demo on wheels had good respons...,Not Available
...,...,...,...,...,...,...,...,...
83,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony A7M4 sellout is good in the market.,Not Available
84,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony New model A7R5 stocks reached outlets las...,Not Available
85,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony APS-C stocks likely to hit market post 15...,Not Available
86,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Nikon Z8 stocks are available in market but no...,Not Available


In [16]:
df.to_csv('.\\Canon Open Source Data\\knowledgeBase.csv')

In [17]:
import mysql.connector as connection 

try:
    mydb=connection.connect(host="localhost",user="root",password='1320')
    print(mydb.is_connected())                                   
    #mydb.close()
except Exception as e:
    print(str(e))

True


In [3]:
# creating a new database 

query= "create database CanonDB"
cursor=mydb.cursor()           
cursor.execute(query)          
print("Database created !")

Database created !


In [18]:
try:
    mydb=connection.connect(host="localhost",database="canonDB",user="root",password="1320",use_pure=True)
    print(mydb.is_connected())                                   # To check if connection is established with the Database
    #mydb.close()
except Exception as e:
    print(str(e))

True


In [8]:
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
cursor=db.cursor()
query = "Drop table canon"
cursor.execute(query)
db.commit()

In [19]:
# Step 4 : Inserting data into the Table

db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
cursor=db.cursor()

# cols = "`,`".join(['ID']+[str(i) for i in df.columns.tolist()])
colvalues=[]
for i,row in df.iterrows():
    colvalues.append(tuple([i+1]+list(row)))

# Insert DataFrame recrds one by one.
for i in colvalues:
    try:
        query = "INSERT INTO CANON VALUES{}".format(i)
        cursor.execute(query)
        db.commit()
    except Exception as e:
        print(e)

In [20]:
# Step5 : Retreiving the same data from the MYSQLdatabase 
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
query="Select * from Canon"
df=pd.read_sql(query,db)
df.head(3)

C:\Users\krish\AppData\Local\Temp\ipykernel_24580\3438997778.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,db)


,index,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
0,2,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
1,3,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
2,4,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available


In [21]:
df.dtypes

index                              int64
Date                              object
Department                        object
Product                           object
Zone                              object
State                             object
Category                          object
Competetor_Product_Information    object
Attachment                        object
dtype: object

In [1]:
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
query="drop table canon"
df=pd.read_sql(query,db)
df.head(3)

In [5]:
current_path = os.getcwd()
media_path = 
os.rmdir('media')

OSError: [WinError 145] The directory is not empty: 'media'

In [2]:
import pandas as pd 
from dateutil.parser import parse
df = pd.read_csv('.\\Canon Open Source Data\\knowledgeBase.csv')
df['Date'] = [parse(df.loc[i].Date) for i in range(df.shape[0])]
# df.drop('Unnamed: 0', axis=1, inplace=True)
df

,index,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
0,0,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
1,1,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
2,2,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available
3,3,2023-06-29,PBC,ICB,East,#VALUE!,Product,Nikon & Sony started promoting Z8 and Zve1 vis...,Not Available
4,4,2023-06-22,CBC-PPP,PPP,India,#VALUE!,Market footfall,Konica Minolta demo on wheels had good respons...,Not Available
...,...,...,...,...,...,...,...,...,...
82,82,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony A7M4 sellout is good in the market.,Not Available
83,83,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony New model A7R5 stocks reached outlets las...,Not Available
84,84,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony APS-C stocks likely to hit market post 15...,Not Available
85,85,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Nikon Z8 stocks are available in market but no...,Not Available


In [9]:
df.Product.value_counts()

ICB      37
PPP      24
CS       23
Laser     3
Name: Product, dtype: int64

In [12]:
df.loc[df.Date=="30-Jun-23"].Zone.value_counts()

North    12
India     3
South     2
Name: Zone, dtype: int64

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

In [3]:
class SQLQuery:
    def __init__(self):
        self.DB_USERNAME = os.getenv('DB_USERNAME')
        self.DB_PASSWORD = os.getenv('DB_PASSWORD')
        self.DB_HOST = os.getenv('DB_HOST')
        self.DB_NAME = os.getenv('DB_NAME')

    def createDBConnectionString(self):
        db_user = self.DB_USERNAME
        db_Password  = self.DB_PASSWORD
        db_host = self.DB_HOST
        db_name = self.DB_NAME
        connectionString = f"mysql+pymysql://{db_user}:{db_Password}@{db_host}/{db_name}"
        return connectionString
    
    def getSQLSchema(self):
        sql_query = f"""  
        SELECT C.TABLE_NAME, C.COLUMN_NAME, C.DATA_TYPE, T.TABLE_TYPE, T.TABLE_SCHEMA  
        FROM INFORMATION_SCHEMA.COLUMNS C  
        JOIN INFORMATION_SCHEMA.TABLES T ON C.TABLE_NAME = T.TABLE_NAME AND C.TABLE_SCHEMA = T.TABLE_SCHEMA  
        WHERE T.TABLE_SCHEMA = '{self.DB_NAME}' 
        """ 
        mysql_connection_string = self.createDBConnectionString()
        result = pd.read_sql_query(sql_query, mysql_connection_string)
        df = result.infer_objects()
        output=[]
        current_table = ''  
        columns = []  
        for index, row in df.iterrows():
            table_name = f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}"  
            column_name = row['COLUMN_NAME']  
            data_type = row['DATA_TYPE']  
            if " " in table_name:
                table_name= f"[{table_name}]" 
            column_name = row['COLUMN_NAME']  
            if " " in column_name:
                column_name= f"[{column_name}]" 
                # If the table name has changed, output the previous table's information  
            if current_table != table_name and current_table != '':  
                output.append(f"table: {current_table}, columns: {', '.join(columns)}")  
                columns = []  
            
            # Add the current column information to the list of columns for the current table  
            columns.append(f"{column_name} {data_type}")  
            
            # Update the current table name  
            current_table = table_name  

        # Output the last table's information  
        output.append(f"table: {current_table}, columns: {', '.join(columns)}")
        output = "\n ".join(output)

        return output

In [7]:
obj = SQLQuery()
conn_string = obj.createDBConnectionString()
df = obj.getSQLSchema()
df

'table: canondb.canon, columns: index int, Date date, Department varchar, Product varchar, Zone varchar, State varchar, Category varchar, Competetor_Product_Information varchar, Attachment varchar'

## VectorDB

In [1]:
import os 
import time
import pinecone as pc
from dotenv import load_dotenv
from langchain.vectorstores import FAISS, Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

load_dotenv()

class Data:
    def __init__(self, pdf_data_path, vector_db_path):
        self.pdf_data_path = pdf_data_path
        self.vector_db_path = vector_db_path
        self.pinecone_api_key = os.getenv('PINECONE_KEY')
        self.pinecone_env = os.getenv('PINECONE_ENV')
        self.embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        self.openai_embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002',
                                                       openai_api_key=os.getenv('OPENAI_KEY'))

    def createPDFVectorDBwithFAISS(self):
        loader = DirectoryLoader(self.pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
        docs = loader.load()
        # text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=400, 
        #                                                 separators=["\n\n", "\n", " ", ""])
        # texts = text_splitter.split_documents(docs)
        db = FAISS.from_documents(docs, self.embedding_model)
        db.save_local(self.vector_db_path)

    def create_top_k_chunk_from_FAISS(self, question, top_k):
        test_idex = FAISS.load_local(self.vector_db_path,self.embedding_model)
        top_k_chunks  = test_idex.similarity_search(question,k=top_k)
        return top_k_chunks
    

    def createPDFVectorDBwithPinecone(self):
        loader = DirectoryLoader(self.pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
        docs = loader.load()
        text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=100, 
                                                        separators=["\n\n", "\n", " ", ""])
        texts = text_splitter.split_documents(docs)
        embeddings = []
        ids = []
        metadatas = []
        for i in range(len(texts)):
            if i%5==0:
                time.sleep(5)
            page_content = texts[i].page_content
            source_pdf = texts[i].metadata['source'].split('\\')[-1]
            page_number = str(texts[i].metadata['page'])
            embedded_page_content = self.openai_embedding_model.embed_query(page_content)
            metadata = {
                'source' : source_pdf,
                'page' : page_number,
                'page_content' : page_content
            }
            ids.append(str(i))
            embeddings.append(embedded_page_content)
            metadatas.append(metadata)

        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        index.upsert(vectors = zip(ids, embeddings, metadatas))


    def create_top_k_chunk_from_Pinecone(self, question, top_k):
        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        #embedded_question = self.embedding_model.embed_query(question)
        vectorstore = Pinecone( index, self.openai_embedding_model.embed_query, text_key='page_content')
        top_k_chunks = vectorstore.similarity_search(question, k=top_k)
        #result = index.query(embedded_question, top_k = top_k, includeMetaData=True)
        return top_k_chunks

c:\Users\krish\anaconda3\envs\project_env\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pdf_data_path = ".\\media"
pdf_vector_embedding_path = ".\\VectorDB"
data_obj = Data(pdf_data_path,pdf_vector_embedding_path)
#data_obj.createPDFVectorDBwithPinecone()
test_question = "Find the cost of Sony ZV-E1 Full Frame camera"
result = data_obj.create_top_k_chunk_from_Pinecone(test_question, top_k =3)
# print(result[0].metadata['source'])
# print(result[0].metadata['page'])

c:\Users\krish\anaconda3\envs\project_env\lib\site-packages\langchain\vectorstores\pinecone.py:61: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Model

In [5]:
import os 
import re
import openai
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory,ConversationBufferMemory,ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

load_dotenv()

class Model:
    def __init__(self):
        self.openai_api_key = os.getenv("OPENAI_KEY")
        self.model = 'text-davinci-003'

    def createQuestionPrompt(self, top_k_chunks):
        prompt= ""
        prompt += 'search results:\n\n'
        for i in range(len(top_k_chunks)):
            meta_info = '['+top_k_chunks[i].metadata['source'].split('\\')[-1] + " , page : " + str(top_k_chunks[i].metadata['page'])+'] '
            page_content = top_k_chunks[i].page_content.replace('\n', ' ')
            page_content = re.sub('\s+', ' ', page_content)
            combined_content = meta_info + page_content
            prompt += combined_content +'\n\n'
        prompt += '''Instructions: Compose a comprehensive reply to the query using the search results given.
                    Cite each reference using [pdfname.pdf , page : number] notation (every result has this number at the beginning).
                    Citation should be done at the end of each sentence. If the search results mention multiple subjects
                    with the same name, create separate answers for each. Only include information found in the results and
                    don't add any additional information. Make sure the answer is correct and don't output false content.
                    If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'
                    Directly start the answer.\n'''.replace('\\n',' ')
        # prompt+= f"Query : {question} \n\n"

        prompt_template = ChatPromptTemplate.from_messages([
                    SystemMessage(content=f"You are a QnA Chatbot whose job is to greet the user politely and asnwer to the question they ask."+prompt), 
                    MessagesPlaceholder(variable_name="chat_history"),         # Where the memory will be stored.
                    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
                 ])
        return prompt_template

    def generateAnswer(self, prompt):
        openai.api_key = self.openai_api_key
        completions = openai.Completion.create(
            engine=self.model,
            prompt=prompt,
            max_tokens=1024,
            temperature=0,
        )
        answer = completions.choices[0]['text']
        return answer
    
    def generateAnswerwithMemory(self,question,prompt_template):
        memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=5)
        llm = ChatOpenAI( openai_api_key = os.getenv("OPENAI_KEY"))
        llm_chain = LLMChain(
            llm=llm,
            prompt=prompt_template,
            verbose=True,
            memory=memory,
        )
        response = llm_chain.predict(human_input = question)
        return response



In [6]:
model_obj = Model()
top_k_chunks = result
prompt_template = model_obj.createQuestionPrompt(top_k_chunks=top_k_chunks)
test_question = "Find the cost of Sony ZV-E1 Full Frame camera"
ans = model_obj.generateAnswerwithMemory(question = test_question, prompt_template=prompt_template)

KeyError: 'page'

In [90]:
ans

'The cost of the Sony ZV-E1 Full Frame camera is MRP: Rs. 214,990. However, it is currently available at an Amazon Offer Price of Rs. 1,88,990 [Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf, page: 0].'

In [83]:
test_question = "Find cons of the camera"
ans = model_obj.generateAnswerwithMemory(question = test_question, prompt_template=prompt_template)




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a QnA Chatbot whose job is to greet the user politely and asnwer to the question they ask.search results:

[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 1]  Technical Details: Brand Sony Manufacturer Sony, Sony Corporation, 1 -7-1 Konan, Minato -KU, Tokyo 108 -0075, Japan Model ZV-E1/BQ IN5 Model Name ZV-E1 Model Year 2023 Product Dimensions 5.43 x 12.1 x 7.19 cm; 483 Grams Batteries 1 Lithium Ion batteries required. (included) Item model number ZV-E1/BQ IN5 Memory Slots Available 1 Compatible Devices Laptop, Desktop, Tablet, Smartphone Special Features Face Detection Mounting Hardware Camera, Rechargeable Battery NP -FZ100, Shoulder strap, Wind Screen, Wind Screen Adapter, Body cap, Accessory shoe cap Number of items 1 Standing screen display size 

[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 0] Sony ZV -E1 Full -Frame Interchangeable -Lens 

In [84]:
ans

"Cons of the Sony ZV-E1 camera include the lack of a viewfinder, which may make it less suitable for users who prefer to compose their shots using a viewfinder [Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf, page: 5]. The camera's ergonomics may not be the best, particularly for users with larger hands [Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf, page: 5]. Additionally, the camera may experience overheating issues [Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf, page: 5]."

In [85]:
test_question = "Provide me a summary of the user reviews ?"
ans = model_obj.generateAnswerwithMemory(question = test_question, prompt_template=prompt_template)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a QnA Chatbot whose job is to greet the user politely and asnwer to the question they ask.search results:

[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 1]  Technical Details: Brand Sony Manufacturer Sony, Sony Corporation, 1 -7-1 Konan, Minato -KU, Tokyo 108 -0075, Japan Model ZV-E1/BQ IN5 Model Name ZV-E1 Model Year 2023 Product Dimensions 5.43 x 12.1 x 7.19 cm; 483 Grams Batteries 1 Lithium Ion batteries required. (included) Item model number ZV-E1/BQ IN5 Memory Slots Available 1 Compatible Devices Laptop, Desktop, Tablet, Smartphone Special Features Face Detection Mounting Hardware Camera, Rechargeable Battery NP -FZ100, Shoulder strap, Wind Screen, Wind Screen Adapter, Body cap, Accessory shoe cap Number of items 1 Standing screen display size 

[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 0] Sony ZV -E1 Full -Frame Interchangeable -Lens 

In [16]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
test_idex = FAISS.load_local(pdf_vector_embedding_path,embedding_model)
test_idex.as_retriever()
llm = ChatOpenAI( openai_api_key = os.getenv("OPENAI_KEY"))
memory = ConversationSummaryMemory(llm=llm, memory_key='chat_history',
                                    return_messages=True,output_key='answer')

qa = ConversationalRetrievalChain.from_llm(llm, test_idex.as_retriever(), return_source_documents=True, 
                                           memory=memory, get_chat_history=lambda h :h,)

In [19]:
def user(query, chat_history):
    chat_history_tuples = []
    for message in chat_history:
        chat_history_tuples.append((message[0], message[1]))

    # Get result from QA chain
    result = qa({"question": query, "chat_history": chat_history_tuples})

    # Append user message and response to chat history
    chat_history.append((query, result["answer"]))

    return chat_history

In [20]:
chat_history = []
query = "What is the price of Sony ZV E1 Camera?"
result = user(query,chat_history=chat_history)

In [22]:
query = "What is the price during offers at Amazon?"
result = user(query,chat_history=result)

In [23]:
result

[('What is the price of Sony ZV E1 Camera?',
  'The Sony ZV-E1 camera is priced at MRP: Rs. 214,990, but it is available at an offer price of Rs. 1,88,990 on Amazon. [Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf, page:1]'),
 ('What is the price during offers at Amazon?',
  'The price of the Sony ZV-E1 camera during the Amazon offer is Rs. 1,88,990.')]

In [36]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"),         # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])
    
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=3)
llm = ChatOpenAI( openai_api_key = os.getenv("OPENAI_KEY"))
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [41]:
res =  llm_chain.predict(human_input = 'Hi, I am KD. What is your name ?')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi, what was my name ?
AI: Your name is KD.
Human: Hi, what was my name ?
AI: I apologize if I made a mistake in my previous responses. As an AI, I don't have access to personal information unless you provide it to me. Therefore, I don't know your name unless you tell me. If you mentioned your name earlier in our conversation, I may have missed it. Could you please remind me of your name?
Human: Hi, I am KD. What is your name ?

> Finished chain.


'Hello KD! Nice to meet you. As an AI language model developed by OpenAI, I don\'t have a personal name. You can simply call me "Assistant." How can I assist you today, KD?'

In [38]:
llm_chain.predict(human_input = 'Hi, what was my name ?')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi, I am KD. What is your name ?
AI: Hello KD! I'm an AI chatbot, so I don't have a physical form or a personal name. You can simply refer to me as "Chatbot." How can I assist you today?
Human: Hi, what was my name ?

> Finished chain.


'Your name is KD. You mentioned it in your initial message. Is there anything else you would like to discuss or inquire about?'

In [39]:
llm_chain.predict(human_input = 'Hi, what was my name ?')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi, I am KD. What is your name ?
AI: Hello KD! I'm an AI chatbot, so I don't have a physical form or a personal name. You can simply refer to me as "Chatbot." How can I assist you today?
Human: Hi, what was my name ?
AI: Your name is KD. You mentioned it in your initial message. Is there anything else you would like to discuss or inquire about?
Human: Hi, what was my name ?

> Finished chain.


'Your name is KD.'

In [40]:
llm_chain.predict(human_input = 'Hi, what was my name ?')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi, what was my name ?
AI: Your name is KD. You mentioned it in your initial message. Is there anything else you would like to discuss or inquire about?
Human: Hi, what was my name ?
AI: Your name is KD.
Human: Hi, what was my name ?

> Finished chain.


"I apologize if I made a mistake in my previous responses. As an AI, I don't have access to personal information unless you provide it to me. Therefore, I don't know your name unless you tell me. If you mentioned your name earlier in our conversation, I may have missed it. Could you please remind me of your name?"

In [35]:
llm_chain.predict(human_input = 'Hi, what was my name ?')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi, what was my name ?
AI: 
AI: Your name is KD.
Human: Hi, what was my name ?
AI: 
AI: Your name is KD. Is there something else I can help you with?
Human: Hi, what was my name ?

> Finished chain.


'\nAI:\nYour name is KD. Is there something else I can help you with, or do you need me to repeat your name again?'

## Splitter

In [17]:
def createQuestionPrompt(question, top_k_chunks):
        prompt= ""
        prompt += 'search results:\n\n'
        for i in range(len(top_k_chunks)):
            meta_info = '['+top_k_chunks[i].metadata['source'].split('\\')[-1] + " , page : " + str(top_k_chunks[i].metadata['page'])+'] '
            page_content = top_k_chunks[i].page_content.replace('\n', ' ')
            page_content = re.sub('\s+', ' ', page_content)
            combined_content = meta_info + page_content
            prompt += combined_content +'\n\n'
        prompt += "Instructions: Compose a comprehensive reply to the query using the search results given."\
                    "Cite each reference using [pdfname.pdf , page : number] notation (every result has this number at the beginning)."\
                    "Citation should be done at the end of each sentence. If the search results mention multiple subjects"\
                    "with the same name, create separate answers for each. Only include information found in the results and"\
                    "don't add any additional information. Make sure the answer is correct and don't output false content."\
                    "If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'"\
                    "Directly start the answer.\n"
        prompt+= f"Query : {question} \n\n"
        return prompt

In [18]:
question = "Find the cost of Sony ZV-E1 Full Frame camera"
createQuestionPrompt(question, top_k_chunks)

'search results:\n\n[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 1] Technical Details: Brand Sony Manufacturer Sony, Sony Corporation, 1 -7-1 Konan, Minato -KU, Tokyo 108 -0075, Japan Model ZV-E1/BQ IN5 Model Name ZV-E1 Model Year 2023 Product Dimensions 5.43 x 12.1 x 7.19 cm; 483 Grams Batteries 1 Lithium Ion batteries required. (included) Item model number ZV-E1/BQ IN5 Memory Slots Available 1 Compatible Devices Laptop, Desktop, Tablet, Smartphone Special Features Face Detection Mounting Hardware Camera, Rechargeable Battery NP -FZ100, Shoulder strap, Wind Screen, Wind Screen Adapter, Body cap, Accessory shoe cap Number of items 1 Standing screen display size\n\n[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 0] Sony ZV -E1 Full -Frame Interchangeable -Lens Mirrorless vlog Camera Working with the ZV -E1\'s 12.1 MP2 full -frame sensor, the latest BIONZ XR ™ engine provides high - speed processing to support 4K 60p, 120

In [5]:
class Data:
    def __init__(self, pdf_data_path, vector_db_path):
        self.pdf_data_path = pdf_data_path
        self.vector_db_path = vector_db_path
        self.pinecone_api_key = os.getenv('PINECONE_KEY')
        self.pinecone_env = os.getenv('PINECONE_ENV')
        self.embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        self.openai_embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002',
                                                       openai_api_key=os.getenv('OPENAI_KEY'))

    def createPDFVectorDBwithFAISS(self):
        loader = DirectoryLoader(self.pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
        docs = loader.load()
        text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=400, 
                                                        separators=["\n\n", "\n", " ", ""])
        texts = text_splitter.split_documents(docs)
        db = FAISS.from_documents(texts, self.embedding_model)
        db.save_local(self.vector_db_path)

    def create_top_k_chunk_from_FAISS(self, question, top_k):
        test_idex = FAISS.load_local(self.vector_db_path,self.embedding_model)
        top_k_chunks  = test_idex.similarity_search(question,k=top_k)
        return top_k_chunks

In [7]:
pdf_data_path = ".\\media"
pdf_vector_embedding_path = ".\\VectorDB"
loader = DirectoryLoader(pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
docs = loader.load()


100%|██████████| 9/9 [00:01<00:00,  7.65it/s]


In [14]:
docs[0].page_content[-200:]

'r warranty. Additional Warranty packs \navailiable ( 3yr & 5yr packs)  \nPrice:  \nMRP: Rs. 26,581     Offer Price: Rs. 23,273 (Includes GST of Rs. 3550)  \nTechnical Details:  \nBrand  \nHP \nManufacturer  '

In [16]:
docs[1].page_content[:200]

'HP , Futaihua Precision Industry (Weihai) Co. Ltd., Jinzhou Road 29#, Hi -Tech IDZ, Weihai, Shangdong  \nSeries  \nHP LASER MFP 1188FNW PRINTER  \nColour  \nWhite  \nForm Factor  \nAll-in-One \nItem Height  '

In [8]:
docs[0].page_content.replace('\n','')

'HP Laser MFP 1188fnw Printer  About this item:  Printer type: Laser; Functions: Print, Copy, Scan, Fax; Printer output: Monochrome; Connectivity: USB/ WiFi -Direct, airprint supported - No; Scanner - Yes; Scanner resolution - Upto 600 dpi  KEY SPECS : Print, Copy, Scan, Fax Wireless, Direct Wifi USB 2.0 H igh Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages, 40 page ADF  OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server, m acOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan; Mobile connectivity - Yes; Hardware Interface - Ethernet, USB 2.0; Enlarge/reduce option - No; Duplex - Auto  Maximum Print Speed (Monochrome) - 21 pages/min; Print c ost Monochrome - INR 3.4; Maximum Print Resolution (Monochrome) - 720 dpi  "Printer Page size: A4, B5, Letter, Legal; Maximum Input Sheet Capacity: 150 sheet input,100 sheet ou

In [96]:
text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 4000, chunk_overlap=200, 
                                                        separators=["\n\n", "\n", " ", ""])
texts = text_splitter.split_documents(docs)
texts[0].page_content

'AccurioPrint C4065 Copier  \nThe AccurioPrint C4065 offers high productivity and media versatility as a compact and affordable \nunit. Perfect for lower volume requirements, it offers professional image and colour quality, media \nversatility and fast upto 66 ppm (A4, Colour), 81 ppm (A4,  black & white).  \n \nAbout this item:  \nDurable Performance:  \nUp to 66 A4 color pages per minute  \nUp to 37 A3 color pages per minute  \n2-sided banner printing, excellent image quality and so much more, the AccurioPrint C4065 from \nKonica Minolta truly does it all i n a compact printing footprint. This revolutionary color production \npress printer has all the features to your business needs to easily develop new revenue streams and \nexpand into new markets.  \n \nReady -made print products:  \nStacking of up to 4,200 sheets  \nTwo-point and corner stapling of up to 100 sheets with automatic staple cut mechanism  \nBooklet Making up to 80 pages (20 sheets)  \n2- and 4 -hole punching  \nTri-F

In [13]:
len(texts[0].page_content)

2695

In [67]:
len(docs[0].page_content)

2421

In [80]:
import fitz
import re
import numpy as np
from tqdm.auto import tqdm


## Input : KEY, pdf path , question
## output : answer

class Data:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path 
        self.start_page = 1
        self.end_page = None

    def _preprocess(self,text):
        '''
        preprocess chunks
        1. Replace new line character with whitespace.
        2. Replace redundant whitespace with a single whitespace
        '''
        text = text.replace('\n', ' ')
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\\u[e-f][0-9a-z]{3}',' ', text)
        return text
    
    def _pdf_to_text(self):
        '''
            convert pdf to a list of words.
        '''
        doc = fitz.open(self.pdf_path)
        total_pages= doc.page_count

        if self.end_page is None:
            self.end_page = total_pages
        text_list=[]

        for i in tqdm(range(self.start_page-1, self.end_page)):
            text= doc.load_page(i).get_text('text')
            text= self._preprocess(text)
            text_list.append(f'[page:{i+1}] '+text)
        doc.close()
        return text_list
    

    def text_to_chunk(self):
        ''''
            converts the text into smaller chunks of word_length.
        '''
        word_length= self.word_length
        texts = self._pdf_to_text()
        tokens = [text.split(' ') for text in texts]
        chunks=[]
        for idx, words in enumerate(tokens):
            for i in range(0,len(words), word_length):
                chunk = words[i:i+word_length]
                if (i+word_length) > len(words) and (len(chunk) < word_length) and (len(tokens) != (idx+1)):
                    tokens[idx+1] = chunk + tokens[idx+1]
                    continue
                chunk = ' '.join(chunk).strip()
                chunk=  f'[{idx+self.start_page}]'+' '+'"'+chunk+'"'
                chunks.append(chunk)
        return chunks

In [81]:
loader = DirectoryLoader('.\\media', glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
docs = loader.load()

100%|██████████| 9/9 [00:01<00:00,  8.97it/s]


In [82]:
pdf_path = ".\\media\\HP Laser 1008w Printer.pdf"
d_obj = Data(pdf_path)
text_list = d_obj._pdf_to_text()
text_list

100%|██████████| 3/3 [00:00<00:00, 462.51it/s]


['[page:1] HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready), 1.1 watts (Sleep), 0.

In [85]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 200,
)

texts = text_splitter.split_documents(docs)
print(len(texts[0].page_content))
print(len(texts[1].page_content))
print(len(texts[2].page_content))

2419
679
1063


In [74]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 200,)

t = text_splitter.create_documents(text_list)

print(len(t[0].page_content))
print(len(t[1].page_content))
print(len(t[2].page_content))

2349
618
1005


In [89]:
t[0].page_content

'HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready), 1.1 watts (Sleep), 0.2 watts (M

https://github.com/langchain-ai/langchain/blob/659817834369799ff7dfba62d79ae6b9e7e8ec7e/libs/langchain/langchain/text_splitter.py#L149

https://python.langchain.com/docs/modules/data_connection/document_transformers/

In [12]:
text_list

['HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready), 1.1 watts (Sleep), 0.2 watts (

In [93]:
import copy
from langchain.docstore.document import Document

class CustomTextSplitter:

    def __init__(self, chunk_size, chunk_overlap):
        self.keep_separator= False
        self.strip_whitespace=True
        self.is_separator_regex=False
        self.add_start_index=False
        self.length_function = len
        self.chunk_size=chunk_size
        self.chunk_overlap = chunk_overlap
        self.separators=["\n\n", "\n", " ", ""]

    def _split_text_with_regex(self,text: str, separator: str):
        # Now that we have the separator, split the text
        if separator:
            if self.keep_separator:
                # The parentheses in the pattern keep the delimiters in the result.
                _splits = re.split(f"({separator})", text)
                splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
                if len(_splits) % 2 == 0:
                    splits += _splits[-1:]
                splits = [_splits[0]] + splits
            else:
                splits = re.split(separator, text)
        else:
            splits = list(text)
        return [s for s in splits if s != ""]
    
    def _join_docs(self, docs, separator:str,):
        text = separator.join(docs)
        if self.strip_whitespace:
            text = text.strip()
        if text == "":
            return None
        else:
            return text
        

    def _merge_splits(self, splits, separator: str):
        # We now want to combine these smaller pieces into medium size
        # chunks to send to the LLM.
        separator_len = self.length_function(separator)

        docs = []
        current_doc= []
        total = 0
        for d in splits:
            _len = self.length_function(d)
            if (
                total + _len + (separator_len if len(current_doc) > 0 else 0)
                > self.chunk_size
            ):
                if total > self.chunk_size:
                    print(
                        f"Created a chunk of size {total}, "
                        f"which is longer than the specified {self.chunk_size}"
                    )
                if len(current_doc) > 0:
                    doc = self._join_docs(current_doc, separator)
                    if doc is not None:
                        docs.append(doc)
                    # Keep on popping if:
                    # - we have a larger chunk than in the chunk overlap
                    # - or if we still have any chunks and the length is long
                    while total > self.chunk_overlap or (
                        total + _len + (separator_len if len(current_doc) > 0 else 0)
                        > self.chunk_size
                        and total > 0
                    ):
                        total -= self.length_function(current_doc[0]) + (
                            separator_len if len(current_doc) > 1 else 0
                        )
                        current_doc = current_doc[1:]
            current_doc.append(d)
            total += _len + (separator_len if len(current_doc) > 1 else 0)
        doc = self._join_docs(current_doc, separator)
        if doc is not None:
            docs.append(doc)
        return docs
    
    def _split_text(self,text: str,):
        """Split incoming text and return chunks."""
        final_chunks = []
        # Get appropriate separator to use
        separator = self.separators[-1]
        new_separators = []
        for i, _s in enumerate(self.separators):
            _separator = _s if self.is_separator_regex else re.escape(_s)
            if _s == "":
                separator = _s
                break
            if re.search(_separator, text):
                separator = _s
                new_separators = self.separators[i + 1 :]
                break

        _separator = separator if self.is_separator_regex else re.escape(separator)
        splits = self._split_text_with_regex(text, _separator)

        # Now go merging things, recursively splitting longer texts.
        _good_splits = []
        _separator = "" if self.keep_separator else separator
        for s in splits:
            if self.length_function(s) < self.chunk_size:
                _good_splits.append(s)
            else:
                if _good_splits:
                    merged_text = self._merge_splits(_good_splits, _separator)
                    final_chunks.extend(merged_text)
                    _good_splits = []
                if not new_separators:
                    final_chunks.append(s)
                else:
                    other_info = self._split_text(s, new_separators)
                    final_chunks.extend(other_info)
        if _good_splits:
            merged_text = self._merge_splits(_good_splits, _separator)
            final_chunks.extend(merged_text)
        return final_chunks
    

    def create_documents(self, texts, metadatas= None):
        """Create documents from a list of texts."""
        # _metadatas = metadatas or [{}] * len(texts)
        # Lst=[]
        documents = []
        for i, text in enumerate(texts):
            index = -1
            for chunk in self._split_text(text):
                info_dict = {}
                #metadata = copy.deepcopy(_metadatas[i])
                if self.add_start_index:
                    index = text.find(chunk, index + 1)
                    #metadata["start_index"] = index
                #info_dict['metadata'] = metadata
                #info_dict['page_content'] = chunk
                #Lst.append(info_dict)
                new_doc = Document(page_content=chunk, metadata={})
                documents.append(new_doc)
                #new_doc = (page_content=chunk, metadata=metadata)
                #documents.append(new_doc)
        return documents
    
    

In [95]:
splitter = CustomTextSplitter(chunk_size=3000, chunk_overlap=200)
splitter.create_documents([".".join(text_list)])

[Document(page_content='[page:1] HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready),

In [87]:
tl=[".".join(text_list)]

In [88]:
create_documents(tl)#[0]['page_content']

[Document(page_content='[page:1] HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready),

In [46]:
_split_text('.'.join(text_list))

['HP Laser 1008w Printer About this item: Printer type: Laser; Functions: Print; printer output: Monochrome; Connectivity (USB/ WiFi); Airprint supported (No); Scanner: No KEY SPECS : Print, Copy, Scan Wireless, Direct Wifi USB 2.0 High Speed connectivity/Ethernet Space saving design 150 sheet input,100 sheet output Duty Cycle - 10000 Pages OS Compatibility: Windows: 7 (32/64 bit), 2008 Server R2, 8 (32/64 bit), 8.1 (32/64 bit), 10 (32/64 bit), 2012 Server, 2016 Server,macOS v10.14 Mojave, macOS v10.13 High Sierra, macOS v10.12 Sierra, OS X v10.11 El Capitan[13](Windows 7 or higher)| Mobile connectivity: Yes; Hardware Interface: USB 2.0; Enlarge/reduce option: No; Duplex: Manual Maximum Print Speed (Monochrome): 21 Pages/min; Print cost Monochrome: 3.4; Maximum Print Resolution (Monochrome): 600 x 600 DPI Printer Page size: A4, Letter, Legal; Maximum Input Sheet Capacity: 150 sheets; Power wattage of printer: 320 watts (Active Printing), 33 watts (Ready), 1.1 watts (Sleep), 0.2 watts (

In [41]:
len(_split_text(text_list[0])[0])

2349

In [50]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(".\\media\\AccurioPrint C4065 Copier.pdf")
pages = loader.load()

In [53]:
texts

[Document(page_content='AccurioPrint C4065 Copier The AccurioPrint C4065 offers high productivity and media versatility as a compact and affordable unit. Perfect for lower volume requirements, it offers professional image and colour quality, media versatility and fast upto 66 ppm (A4, Colour), 81 ppm (A4, black & white). About this item: Durable Performance: Up to 66 A4 color pages per minute Up to 37 A3 color pages per minute 2-sided banner printing, excellent image quality and so much more, the AccurioPrint C4065 from Konica Minolta truly does it all in a compact printing footprint. This revolutionary color production press printer has all the features to your business needs to easily develop new revenue streams and expand into new markets. Ready-made print products: Stacking of up to 4,200 sheets Two-point and corner stapling of up to 100 sheets with automatic staple cut mechanism Booklet Making up to 80 pages (20 sheets) 2- and 4 -hole punching Tri-Fold up to 3 sheets Sophisticated

"Instructions: Compose a comprehensive reply to the query using the product details given.Cite each reference using [pdfname.pdf , page : number] notation (every result has this number at the beginning).Citation should be done at the end of each sentence. If the search results mention multiple subjectswith the same name, create separate answers for each. Only include information found in the results anddon't add any additional information. Make sure the answer is correct and don't output false content.If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'Directly start the answer."